In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys

import tensorflow as tf
import pandas as pd
import tensorflow as tf


In [3]:

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path,
                        names=CSV_COLUMN_NAMES, 
                        header=0  # ignore first row
                       )
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)
        
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset


# The remainder of this file contains a simple example of a csv parser,
#     implemented using a the `Dataset` class.

# `tf.parse_csv` sets the types of the outputs to match the examples given in
#     the `record_defaults` argument.
CSV_TYPES = [[0.0], [0.0], [0.0], [0.0], [0]]

def _parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, record_defaults=CSV_TYPES)

    # Pack the result into a dictionary
    features = dict(zip(CSV_COLUMN_NAMES, fields))

    # Separate the label from the features
    label = features.pop('Species')

    return features, label


def csv_input_fn(csv_path, batch_size):
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(1)

    # Parse each line.
    dataset = dataset.map(_parse_line)

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


In [4]:
BATCH_SIZE=100
TRAIN_STEPS=1000

def main():
    
    # fetch data
    (train_x, train_y), (test_x, test_y) = load_data()
    
    # create feature columns
    feature_columns = []
    for feature in train_x.keys():
        feature_columns.append(tf.feature_column.numeric_column(key=feature))
        
    # build a 2 layer DNN
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        hidden_units=[10, 10],  # two layers, each 10 nodes
        n_classes=3  # 3 classes to pick from
    )
    
    # train the model
    classifier.train(
        input_fn=lambda:train_input_fn(train_x, train_y, BATCH_SIZE),
        steps=TRAIN_STEPS
    )
    
    # evaluate
    eval_result = classifier.evaluate(
        input_fn=lambda:eval_input_fn(test_x, test_y, BATCH_SIZE)
    )
    
    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    
    
    
#tf.logging.set_verbosity(tf.logging.INFO)
main()



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/qk/lr28pg8j4bd19r13ft1c3l2h0000gn/T/tmpupyf_uys', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10dded0b8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/qk/lr28pg8j4bd19r13ft1c3l2h0000gn/T/tmpupyf_uys/model.ckpt.
INFO:tensorflow:loss = 227.45163, step = 1
INFO:tensorflow:global_step/sec: 678.33
INFO:tensorflow:loss = 16.79317, step = 101 (0.149 sec)
INFO:tensorflow:global_step/sec: 784.356
INFO:tensorflow:loss = 8.629014, step